# Classifying Movie Reviews from IMDb

The Internet Movie Database (IMDb) is an online database consisting of movie, TV, and video game information.  Our goal is to analyze a dataset of movie reviews from IMDb and use the Python neural network library [Keras](https://keras.io/) to predict the sentiment of a movie review.  The dataset consists of 25,000 movie reviews from IMDb, labeled by sentiment (positive review or negative review). Reviews have been preprocessed, and each review is encoded as a sequence of word indices (integers).  Words are indexed by overall frequency in the dataset, so for example the integer 1 corresponds to the most frequent word in the dataset, 2 corresponds to the 2nd most frequent word, etc.  Therefore, a sentence is represented by a sequence of integers, and each movie review can be encoded as an integer vector.

For humans, it is easy to predict the sentiment of a review.  This is something we want to train the model to do.

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(13)

Using TensorFlow backend.


## Load and prepare the data

With Keras, this dataset comes [preloaded](https://keras.io/datasets/), so a simple command will allow us to access the training and testing data. The parameter `num_words` specifies the top most frequent words to consider.  The training and testing data are loaded into feature matrices `X_train` and `X_test` (represented as numpy arrays), and the corresponding training/testing labels (`y_train/y_test`) are vectors of binary integers (0 or 1), where 0 represents a bad review and 1 represents a good review.

In [13]:
# Load the data and split it into 50% train, 50% test sets
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

print('number of training examples: ', len(X_train))
print('number of testing examples: ', len(X_test))

number of training examples:  25000
number of testing examples:  25000


We can look at the first training example, which is a movie review encoded as an array of integers

In [14]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


## How to recover the orginal review?

The above array is how the algorithm will see the review, but it would be nice to see the review in human-readable form as well, so we can get an idea of the review's sentiment.  There is a function in Keras call `get_word_index` that returns a dictionary mapping words to their corresponding indices.  For example, we can have a look at (say, 10) random entries from the dictionary with the restriction that these 10 entries come from the top 5000 most frequent words.

In [12]:
# print out 10 random entries from the top 5000 most frequent words in the word_index dict
word_index = imdb.get_word_index()
rand_indices = np.random.randint(5000, size=10)
for k,v in word_index.items():
    if v in rand_indices:
        print(k,v)

under 464
sally 3507
coherent 3981
floor 1870
wont 4280
roger 2474
ken 3659
doctor 1039
sensitive 2725
each 254


We can use this word index and write a function that translates an integer-encoded review back into human-readable form.  For more information, see the first answer to this [stackoverflow post](https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset).  

In [5]:
def get_review(review_number):
    '''
    Put the review back in a form that humans can understand
    '''
    word_index = imdb.get_word_index()
    word_index = {k:v+3 for k, v in word_index.items()}
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2
    index_word = {v:k for k, v in word_index.items()}
    return ' '.join( index_word[i] for i in X_train[review_number] )

# print out the first two reviews and their labels
print('The first review is clearly positive, so it should get a label of {} :\n'.format(y_train[0]))
print(get_review(0), '\n')
print('The second review is clearly negative, so it should get a label of {} :\n'.format(y_train[1]))
print(get_review(1))

The first review is clearly positive, so it should get a label of 1 :

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly <UNK> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big <UNK> for the whol

## Sequence padding

The neural network model requires that the input vectors all have the same length, so we need to set a fixed length for the inputs and then find a way to deal with the movie reviews that are too long or too short.  This can be done using [sequence.pad_sequences](https://keras.io/preprocessing/sequence/) in the `keras.preprocessing` module, which will truncate longer reviews to a fixed length and also pad shorter reviews with zeros.

In [15]:
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

print('train/test set dimensions: ', X_train.shape, X_test.shape, '\n')
print('first sentence in training set, encoded and padded:\n\n ', X_train[0], '\n')

train/test set dimensions:  (25000, 500) (25000, 500) 

first sentence in training set, encoded and padded:

  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   

## Building the model architecture

### 1. Using GridSearchCV to find optimal hyperparameters

In [16]:
def build_model(optimizer='rmsprop', input_dim=5000, output_dim=32, input_length=500):
    '''
    Setup the model architecture and compile the model.
    '''
    model = Sequential()
    
    # input layer
    model.add(Embedding(input_dim, output_dim, input_length=max_review_length))
    
    # first hidden layer 
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    
    # second hidden layer
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    
    # output layer
    model.add(Dense(1, activation='logistic'))
    
    # compile the model
    #optimizer = RMSprop(lr=learn_rate, rho=grad_decay)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    return model

def grid_search(model, param_grid):
    '''
    Perform a grid search of model hyperparameters and return the best model,
    along with a summary of the different grid combinations.
    '''
    # set up and perform the grid search
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result = grid.fit(X_train, y_train)

    # the cv_results_ attribute is a dict that summarizes many important results
    # from each model
    test_score_means = grid_result.cv_results_['mean_test_score']
    train_score_means = grid_result.cv_results_['mean_train_score']
    train_times = grid_result.cv_results_['mean_fit_time']
    params = grid_result.cv_results_['params']

    # store these results in a smaller dict for easy loading into a pandas dataframe
    final_results = dict(mean_test_score=test_score_means, mean_train_score=train_score_means,
                         mean_fit_time=train_times, params=params)

    print('Best score {} using hyperparameters {}'.format(grid_result.best_score_, grid_result.best_params_))
    
    return grid_result, final_results


### Neural Network with LSTM Layer

Recurrent neural networks and LSTMs are a pretty complicated topic in deep learning, and I'm only at the point where I barely understand these topics myself.  One of the dangers with using a high-level library like Keras (in my opinion) is that it can be way too easy to implement these models without having a clue what the hell you're doing.  In order to really understand Machine Learning (and deep learning in particular), it's important to understand the mathematics behind the algorithms and techniques.  A really nice lecture on recurrent neural networks is available [here](https://www.youtube.com/watch?v=6niqTuYFZLQ&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv) (starting at about 9 minutes into the video).  It is from Stanford's [CS 231N Course](http://cs231n.stanford.edu/syllabus.html) on Convolutional Neural Networks for Image Recognition.  There is also a great online deep learning textbook available [here](http://www.deeplearningbook.org/). 

We will build a sequential model with an embedding layer and an LSTM layer. The sequential model is just a linear stack of layers in the neural network.  The first layer is an [Embedding](https://keras.io/layers/embeddings/) layer that maps each integer-encoded word into a space of word vectors where semantically similar words are mapped to nearby points.  This natural language processing model is called [word2vec](https://www.tensorflow.org/tutorials/word2vec).

After the embedding layer, we add an [LSTM](https://keras.io/layers/recurrent/#lstm) layer.  LSTM is an abbreviation for Long Short-Term Memory Unit, and these were originally proposed as a solution to the [vanishing gradient problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem) (and the exploding gradient problem).  In a nutshell, the vanishing/exploding gradient problem are problems when training neural networks with gradient descent methods and back-propagation.  When a neural network is trained, it tries to minimize a cost function by back-propagating the error through the network and repeatedly computing the gradients of composite functions through the layers.  Depending on the activation functions used, the gradients can either become so small or so large that further weight updates become infeasible.  LSTMs help preserve the error that is back-propagated through the network. 

We will start by trying the sigmoid activation function.

In [7]:
# model parameters
top_words = 5000
embedding_dim = 32
max_review_length = 500
activation_function = 'sigmoid'
loss_function = 'binary_crossentropy'
optimization_method = 'adam'
evaluation_metrics = ['accuracy']

def build_lstm_model(input_dim=top_words, output_dim=embedding_dim, input_length=max_review_length,
                activation=activation_function, loss=loss_function,
                optimizer=optimization_method, metrics=evaluation_metrics):
    '''
    Setup the model architecture, compile the model, and give a model summary.
    '''
    model = Sequential()
    model.add(Embedding(input_dim, output_dim, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(1, activation=activation_function))
    model.compile(optimizer, loss, metrics=evaluation_metrics)
    print(model.summary())
    return model

imdb_model = build_model();

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301.0
Trainable params: 213,301
Non-trainable params: 0.0
_________________________________________________________________
None


## Training the model


In [8]:
# more model parameters

def train_model():
    
    pass

## Evaluating the model

Let's see the accuracy of the model, along with the precision and recall.

In [9]:
# score = model.evaluate(x_test, y_test, verbose=0)
# print("Accuracy: ", score[1])